# Notebook to loop through snowline images and select the annual ELA (maximum snowline estimate)

In [ ]:
# Install packages note already in environment
!pip install geopandas rioxarray

In [ ]:
# Import packages
import os
import matplotlib.pyplot as plt
import glob
import pandas as pd
import geopandas as gpd
import numpy as np
import sys
from shapely import wkt
import sys
import json
import sys
import ee
import datetime

In [ ]:
# Authenticate your Google Earth Engine account
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# If using Google Colab, mount Google Drive so you can access the files in this folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Uncomment the line below to locate any needed paths in directory
# os.listdir('drive/MyDrive/Research/PhD/snow_cover_mapping/snow_cover_mapping_application/study-sites/')

In [ ]:
##### MODIFY HERE #####
# specify site name
site_name = 'Gulkana'
# path to snow-cover-mapping-application/
# RAINEY'S PATH:
base_path = 'drive/My Drive/Research/PhD/snow_cover_mapping/snow_cover_mapping_application/snow-cover-mapping-application/'
# ALEXANDRA'S PATH:
# base_path = ('drive/MyDrive/snow_cover_mapping_application/snow-cover-mapping-application/')
# path to study site
study_site_path = base_path + '../study-sites/' + site_name + '/'
# path where figures will be saved
figures_out_path = study_site_path + 'figures/'
# path where manual ELA estimates will be saved
out_path = study_site_path +'imagery/ELAs/'
# define month range for filtering snowlines from file (includes both months)
month_start = 7
month_end = 10
#######################

# -----Determine other folders
# path to PlanetScope image mosaics
PS_im_path = study_site_path + 'imagery/PlanetScope/mosaics/'
# path to snowline files
sl_est_path = study_site_path + 'imagery/snowlines/'
# path and file name of glacier outline shapefile
AOI_fn = glob.glob(study_site_path + 'AOIs/' + '*.shp')[0]

# -----Load dataset characteristics
dataset_dict = json.load(open(base_path + 'datasets_characteristics.json'))

# -----Load functions from file
sys.path.append(base_path)
import manually_select_ELAs_utils as f

In [ ]:
# -----Load and compile snowline files
sl_est_fns = glob.glob(sl_est_path + '*snowline.csv')
# only include snowlines starting in 2016 (time series too sparse before then)
sl_est_fns = [sl_est_fn for sl_est_fn in sl_est_fns if ('2013' not in sl_est_fn)
                                                    & ('2014' not in sl_est_fn)
                                                    & ('2015' not in sl_est_fn)]
# compile all snowline files into one DataFrame
sl_est_full = gpd.GeoDataFrame()
for fn in sl_est_fns:
    # read file
    sl_est_df = pd.read_csv(fn)
    # create a geopandas GeoDataFrame from the DataFrame
    sl_est_df.loc[sl_est_df['geometry']=='[]', 'geometry'] = None
    sl_est_df['geometry'] = sl_est_df['geometry'].apply(wkt.loads)
    sl_est_gdf = gpd.GeoDataFrame(sl_est_df, crs=sl_est_df['CRS'].iloc[0],
                                  geometry=sl_est_df['geometry'])
    # concatenate to df
    sl_est_full = pd.concat([sl_est_full, sl_est_gdf])
# read dates as datettime64, only include snowlines for July - November
sl_est_full['datetime'] = sl_est_full['datetime'].astype('datetime64[ns]')
sl_est_full['month'] = sl_est_full['datetime'].values.astype('datetime64[M]').astype(int) % 12 + 1
sl_est_full = sl_est_full.loc[(sl_est_full['month'] >= month_start) & (sl_est_full['month'] <= month_end)]
# remove month column
sl_est_full = sl_est_full.drop('month', axis=1)
# remove Unnamed column if present
if 'Unnamed: 0' in list(sl_est_full.columns):
    sl_est_full = sl_est_full.drop('Unnamed: 0', axis=1)
# renumber, sort by date
sl_est_full = sl_est_full.sort_values(by=['datetime']).reset_index(drop=True)
# convert evaluate strings in each column
sl_est_full['snowlines_coords_X'] = sl_est_full['snowlines_coords_X'].apply(eval).apply(np.array)
sl_est_full['snowlines_coords_Y'] = sl_est_full['snowlines_coords_Y'].apply(eval).apply(np.array)

# -----Load AOI
AOI = gpd.read_file(AOI_fn)
# reproject to same CRS as snowlines
AOI = AOI.to_crs(sl_est_full['CRS'][0])

sl_est_full

## 1. Select annual ELAs

In [ ]:
checkboxes = f.manual_snowline_filter_plot(sl_est_full, dataset_dict, AOI, PS_im_path)

## 2. View resulting annual ELAs

In [ ]:
# -----Grab ELA selections from checkboxes
values = np.array([x.value for x in checkboxes])
IELAs = np.ravel(np.argwhere(values==True))
ELAs = sl_est_full.iloc[IELAs]

# -----Plot
plt.figure(figsize=(12,6))
plt.plot(sl_est_full['datetime'].astype('datetime64[ns]'), sl_est_full['snowlines_elevs_median_m'], '.k')
plt.plot(ELAs['datetime'].astype('datetime64[ns]'), ELAs['snowlines_elevs_median_m'], '*m', markersize=15)
plt.grid()
plt.ylabel('Median snowline elevation [m]')

## 3. Save annual ELAs to file

When you're satisfied with your ELA selections, save them!

In [ ]:
# define file name and where to save it
out_fn = site_name + '_manually_selected_ELAs.csv'
out_path = study_site_path + 'ELAs/'

# make output directory (out_path) if it doesn't exist
if not os.path.exists(out_path):
  os.mkdir(out_path)
  print('Made directory for outputs: '+out_path)

# save to file
ELAs.to_csv(out_path + out_fn, index=False)
print('ELAs saved to file: ' + out_path + out_fn)